In [1]:
import pandas as pd
import numpy as np

In [7]:
df = pd.read_csv("tennis.txt")
df

,Day,Outlook,Humidity,Wind,Play
0,1,Sunny,High,Weak,No
1,2,Sunny,High,Strong,No
2,3,Overcast,High,Weak,Yes
3,4,Rain,High,Weak,Yes
4,5,Rain,Normal,Weak,Yes
5,6,Rain,Normal,Strong,No
6,7,Overcast,Normal,Strong,Yes
7,8,Sunny,High,Weak,No
8,9,Sunny,Normal,Weak,Yes
9,10,Rain,Normal,Weak,Yes


## Node

In [10]:
class Node():
    """
    A class representing a node in a decision tree
    """

    def __init__(self, feature = None, threshold = None, left = None, right = None, gain = None, value = None):
        """
        Initializes a new instance of the Node class
        
        Args:
            feature: The feature used for splitting at this node. Defaults to None
            threshold: The threshold used for splitting at this node. Defaults to None.
            left: The left child node. Defaults to None
            right: The right child node. Defaults to None
            gain: The gain of the split. Defaults to None
            value: If this node is a leaf node, this attribute represents the predicted value for the target variable. Defaults to None
        """
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.gain = gain
        self.value = value


## DecisionTree class

In [12]:
class DecisionTree():
    """
    A decision tree classifier for binary classification problems.
    """

    def __init__(self, min_samples = 2, max_depth = 2):
        """
        Constructor for DecisionTree class

        Paraneters:
            min_samples (int): Minimum number of samples required to split an internal node
            max_depth (int): Maximum depth of the decision tree.
        """
        self.min_samples = min_samples
        self.max_depth = max_depth

    def split_data(self, dataset, feature, threshold):
        """
        Splits the given dataset into two datasets based on the given feature and threshold

        Parameters:
            dataset(ndarray): Input dataset
            feature (int): Index of the feature to be split on
            threshold(float): Threshold value to split the feature on
        
        Returns:
            left_dataset (ndarray): Subset of the dataset with values less than or equal to the threshold
            right_dataset (ndarray): Subset of the dataset with values greater than the threshold
        """
        # Create empty arrays to store the left and right datasets
        left_dataset = []
        right_dataset = []

        # Loop over each row in the dataset and split based on the given feature and threshold
        for row in dataset:
            if row[feature] <= threshold:
                left_dataset.append(row)
            else:
                right_dataset.append(row)
        
        # Convert the left and right datasets to numpy arrays and return
        left_dataset = np.array(left_dataset)
        right_dataset = np.array(right_dataset)
        return left_dataset, right_dataset

    def entropy(self, y):
        """
        Computes the entropy of the given label values

        Parameters:
            y(ndarray): Input label values

        Returns:
            entropy(float): Entropy of the given label values
        """
        entropy = 0

        # Find the unique label values in y and loop over each value
        labels = np.unique(y)
        for label in labels:
            # Find the examples in y that have the current label
            label_examples = y[y == label]
            # Calculate the ratio of the current label in y
            pl = len(label_examples)/len(y)
            # Calculate the entropy using the current label and ratio
            entropy += -pl * np.log2(pl)
        # Return the final entropy value
        return entropy

    def information_gain(self, parent, left, right):
        """
        Computes the information gain from splitting the parent dataset into two datasets

        Parameters:
            parent (ndarray): Input parent dataset
            left (ndarray): Subset of the parent dataset after split on a feature
            right (ndarray): Subset of the parent dataset after split on a feature

        Returns:
            information_gain (float): Information gain of the split
        """
        # set initial information gain to 0
        information_gain = 0
        # compute entropy for parent
        parent_entropy = self.entropy(parent)
        # calculate weight for left and right nodes
        weight_left = len(left)/len(parent)
        weight_right = len(right)/len(parent)
        # compute entropy for left and right nodes
        entropy_left, entropy_right = self.entropy(left), self.entropy(right)
        # calculate weighted entropy
        weighted_entropy = weight_left * entropy_left + weight_right * entropy_right
        # calculate information gain
        information_gain = parent_entropy - weighted_entropy
        return information_gain

    def best_split(self, dataset, num_samples, num_features):
        """
        Finds the best split for the given dataset

        Args:
        dataset (ndarray): The dataset to split
        num_samples (int): The number of samples in the dataset
        num_features (int): The number of features in the dataset

        Returns:
        dict: A dictionary with the best split feature index, threshold, gain, left and right dataset.
        
        """
        # dictionary to store the best split values
        best_split = {'gain':-1, 'feature': None, 'threshold': None}
        # loop over all the features
        for feature_index in range(num_features):
            #get the feature at the current feature_index
            feature_values = dataset[:, feature_index]
            #get unique values of that feature
            thresholds = np.unique(feature_values)
            # loop over all values of the feature
            for threshold in thresholds:
                # get left and right datasets
                left_dataset, right_dataset = self.split_data(dataset, feature_index, threshold)
                # check if either datasets is empty
                if len(left_dataset) and len(right_dataset):
                    # get y values of the parent and left, right nodes
                    y, left_y, right_y = dataset[:, -1], left_dataset[:, -1], right_dataset[:, -1]
                    # compute information gain based on the y values
                    information_gain = self.information_gain(y, left_y, right_y)
                    # update the best split if conditions are met
                    if information_gain > best_split['gain']:
                        best_split["feature"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["left_dataset"] = left_dataset
                        best_split["right_split"] = right_dataset
                        best_split["gain"] = information_gain
        return best_split

    def calculate_leaf_value(self, y):
        """
        Calculates the most occuring value in the given list of y values

        Args:
            y (list): The list of y values

        Returns:
            The most occuring value in the list.
        """
        y = list(y)
        # get the highest present class in the array
        most_occuring_value = max(y, key=y.count)
        return most_occuring_value

    def build_tree(self, dataset, current_depth=0):
        """
        Recursively builds a decision tree from the given dataset.

        Args:
        dataset (ndarray): The dataset to buld the tree from.
        current_depth (int): The current depth of the tree

        Returns:
        Node: The root node of the built decision tree
        """
        # split the dataset into X, y values
        X, y = dataset[:, :-1], dataset[:, -1]
        n_samples, n_features = X.shape
        # keeps spliting until stopping conditions are met
        if n_samples >= self.min_samples and current_depth <= self.max_depth:
            # Get the best split
            best_split = self.best_split(dataset, n_samples, n_features)
            # Check if gain isn't zero
            if best_split["gain"]:
                # continue splitting the left and the right child. Increment current depth
                left_node = self.build_tree(best_split["left_dataset"], current_depth + 1)
                right_node = self.build_tree(best_split["right_dataset"], current_depth + 1)
                # return decision node
                return Node(best_split["feature"], best_split["threshold"], left_node, right_node, best_split["gain"])
        # compute leaf node value
        leaf_value = self.calculate_leaf_value(y)
        # return leaf node value
        return Node(value=leaf_value)

    def fit(self, X, y):
        """
        Builds and fits the decision tree to the given X and y values.

        Args:
        X (ndarray): The feature matrix
        y (ndarray): The target values
        """
        dataset = np.concatenate((X, y), axis=1)
        self.root = self.build_tree(dataset)

    def predict(self, X):
        """
        Predicts the class labels for each instance in the feature matrix X.

        Args:
        X (ndarray): The feature matrix to make predictions for.

        Returns:
        list: A list of predicted class labels
        """
        # Create an empty list to store the predictions
        predictions = []
        # For each instance in X, make a prediction by traversing the tree
        for x in X:
            prediction = self.make_prediction(x, self.root)
            # Append the prediction to the list of predictions
            predictions.append(prediction)
        # Convert the list to a numpy array and return it
        np.array(predictions)
        return predictions
    
    def make_prediction(self, x, node):
        """
        Traverses the decision tree to predict the target value for the given feature vector

        Args:
        x (ndarray): The feature vector to predict the target value for.
        node (Node): The current node being evaluated.
        """
        # if the node has value i.e it's a leaf node extract it's value
        if node.value != None:
            return node.value
        else:
            # if it's node a leaf node we'll get it's feature and traverse through the tree accordingly
            feature = x[node.feature]
            if feature <= node.threshold:
                return self.make_prediction(x, node.left)
            else:
                return self.make_prediction(x, node.right)


## II. Evaluation:

In [ ]:
def train_test_split(X, y, random_state=41. test_size=0.2):
    """
    Splits the data into training and testing sets.

    Parameters:
        X (numpy.ndarray): Features array of shape (n_samples, n_features)
        y (numpy.ndarray): Target arrray of shape (n_samples)
        random_state (int): Seed for the random number generator. Default is 42
        test_size (float): Proportion of samples to include in the test set. Default is 0.2

    Returns:
        Tuple[numpy.ndarray]: A tuple containing X_train, X_test, y_train, y_test
    """
    # Get number of shape
    n_samples = X.shape[0]

    # Set the random seed for random generator
    np.random.seed(random_state)

    # Shuffle the indices
    shuffle_indices = np.random.permutation(np.arange(n_samples))

    # Determine the size of the test set
    test_size = int(n_samples * test_size)

    # Split the indices into test and train
    test_indices = shuffle_indices[:test_size]
    train_indices = shuffle_indices[test_size:]


In [ ]:
model = DecisionTree(2,2)
model.fit